## Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['Ohio Final Train.ipynb', 'GoogleNews-vectors-negative300.bin.gz', 'COPY!!!!!!!!!!!!!!!!!.ipynb', 'gpt2-gerchef', 'Copy of Fine-tune a non-English GPT-2 Model with Huggingface', 'tess_copy_commentgeneration.ipynb', 'LifeProTips', 'UIUC', 'bigtenLinearSVC.pickle', 'model1.ipynb', 'UIUC2', 'preprocessCornell.ipynb', 'uofm', 'askredditpreprocessing.ipynb', 'gpt-2-with-huggingface-pytorch.ipynb', 'Test_Evaluation_Metrics.ipynb.ipynb', 'commentgeneration.ipynb', 'train_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'test_com_set.pickle', 'todayilearnedgeneration.ipynb', 'val_post_set.pickle', 'val_com_set.pickle', 'COPY FOR TRYING SEQTOSEQ uiucgenerationmanual.ipynb', 'Cornell Final Train.ipynb', 'Illinois', 'UMD', 'Indiana', 'Illinois Final Train.ipynb', 'Iowa', 'msu', 'uofmn', 'Indiana Final Train.ipynb', 'Penn', 'Ohio', 'Penn Final Train.ipynb', 'uofmn Final Train.ipynb', 'lincoln', 'Northwestern', 'rutgers', 'UWMadison', 'Cornell (OLD FOLDER

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [3]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=

In [6]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [8]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [9]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/uiowa.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
# corpus = Corpus(filename=download("subreddit-Cornell"))
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 1739
Number of Utterances: 7945
Number of Conversations: 1532


In [10]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()

In [11]:
temp = utt_df.sort_values("meta.score", ascending=False)
temp.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
9864ly,1534540775,,iamtheawesome10,None,9864ly,93,None,1536770262,0,None,uiowa,False,/r/uiowa/comments/9864ly/i_felt_attacked_durin...,,[]
9e21vk,1536385879,,Vikingboy9,None,9e21vk,89,None,1540093230,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",uiowa,False,/r/uiowa/comments/9e21vk/upvote_if_you_can_read/,,[]
9f418g,1536721030,,tvclassicsif,None,9f418g,86,None,1540110947,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",uiowa,False,/r/uiowa/comments/9f418g/iowa_city_iowa/,,[]
28e25a,1403030065,,dwjohnson31,None,28e25a,83,None,1441307720,0,None,uiowa,False,/r/uiowa/comments/28e25a/we_are_everywhere/,,[]
9dwiju,1536341142,,tvclassicsif,None,9dwiju,78,None,1540090612,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",uiowa,False,/r/uiowa/comments/9dwiju/we_can_all_agree_on_t...,,[]


In [13]:
temp = utt_df.sort_values("meta.score", ascending=False)

posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]

posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]

conversation_ids_in = list(posts["conversation_id"])

coversation_ids = []
num_ids = 0
for id in conversation_ids_in:
  if num_ids == 20000:
      break
  dataframe = comments[comments["conversation_id"] == id]
  com_ids = list(dataframe.index)
  if com_ids != []:
    coversation_ids.append(id)
    num_ids = num_ids + 1

top_posts = posts.head(20000)
top_posts.head()

print(len(top_posts))


780


In [14]:
coversation_ids = coversation_ids[:20000] #not sure if needed 

In [15]:
datasets_to_use = []
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

In [16]:
import re
from nltk import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
train_ids, test_ids = train_test_split(coversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]
for index, id_list in enumerate(ids_total):
  for id in id_list:
      dataframe = comments[comments["conversation_id"] == id]
      post = posts.loc[id]['text']
      post = re.sub("\n", " ", post)
      post = re.sub("&gt;", "", post)
      post = re.sub("  ", " ", post)
      post = re.sub("&amp;", "", post)
      post = re.sub("#x200B;", "", post)
      post = re.sub(r'http\S+', '[url link]', post)
      post_tokens = word_tokenize(post)
      len_post = len(post_tokens)
      post_len[index].append(len_post)
      commented_here = ""
      com_ids = list(dataframe.index)
      if (len_post <= 512):
          running_sum = 0 # testing
          for id_c in com_ids:
            out_text = dataframe.loc[id_c]['text']
            out_text = str(out_text).strip()
            out_text = re.sub(r"\s", " ", out_text)
            out_text = re.sub(r"\n", " ", out_text)
            out_text = re.sub("&gt;", "", out_text) # added preprocess
            out_text = re.sub("  ", " ", out_text) # added preprocess
            out_text = re.sub("&amp;", "", out_text) # added preprocess
            out_text = re.sub("#x200B;", "", out_text) # added preprocess
            out_text = re.sub(r'http\S+', '[url link]', out_text) # added preprocess
            out_text_tokens = word_tokenize(out_text) # added this
            running_sum += len(out_text_tokens) # added this
            commented_here = commented_here + " " + out_text
            if running_sum > len_post: # changed this to 512
              break # added this
          comment_tokens = word_tokenize(commented_here)
          com_len[index].append(len(comment_tokens))
          post_utter = "<bos> " + post + " <eos> "
          comment_utter = "<bos>" + commented_here + " <eos> "
          posts_datasets_to_use[index].append(post_utter)
          comments_datasets_to_use[index].append(comment_utter)
print(posts_datasets_to_use[0][:10])
print(comments_datasets_to_use[0][:10])
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(post_len[0][0])
print(com_len[0][0])
print(len(post_len[0]))
print(len(com_len[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


["<bos> ###You may already know this, but college students with .edu email accounts are eligible for many online perks. I found this list from an AskReddit post a year back and found it surprisingly useful, although a bit outdated. I updated it and narrowed it down to the most worthwhile offers, but you can see the all the responses here: [**AskReddit**]([url link]  **[Amazon Prime]([url link] *6 month free trial, 50% off after trial* * Free unlimited photo storage * FREE Two-Day shipping with Amazon Prime * Watch unlimited movies and TV shows with Prime Instant Video * Shop Amazon Lightning Deals and MyHabit.com * A Prime membership includes over 500,000 free ebooks **[Autodesk]([url link] *Free 3D Design, Engineering  Entertainment Software* * AutoCAD * Maya (3D animation software) * 3ds Max * Revit * Fusion 360 Ultimate * Inventor Professional **[Microsoft Dreamspark]([url link] *Free Microsoft software* * Visual Studio Community 2013 (make applications for PC, Android, or iPhone) *

In [17]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 96))
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 75))

print(averages_post)
print(boundary_post)
print(averages_com)
print(boundary_com)

[76.17463617463618, 78.75, 69.76543209876543]
[206.5999999999999, 222.84, 180.5999999999999]
[100.15723270440252, 138.54430379746836, 98.0]
[119.0, 159.5, 114.0]


In [18]:
train_tensors = []
val_tensors = []
test_tensors = []
train_com_tensors = []
val_com_tensors = []
test_com_tensors = []
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(len(tokenizer))

50260


In [19]:
post_tensors_data = [train_tensors, val_tensors, test_tensors]
comments_tensors_data = [train_com_tensors, val_com_tensors, test_com_tensors]
# there doesn't seem to be a way to associate the two permanently, since tokenizer automatically removes it.
# maybe the exception is you reach into the other comment and remove?
# for instance, loop with like index: if it excepts, then: you go into the other coms dataset_to_use and remove that specific index.
# say in train, which is index = 0, and we find that at index1, post is too long. go into coms_dataset_to_use and delete that entry.
# i assume that it does the same thing for the other one. 
post_max = []
com_max = []
for index, dataset_tensors in enumerate(posts_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    post_tensors_data[index].append(text)
    #except:
    #print(index1) # watch for big tokens how to. like does this already truncate? 
    #del comments_datasets_to_use[index][index1]
    #del posts_datasets_to_use[index][index1] #maybe this one too for the next pass around
      # only problem is are the indexes reliable? like does index1 mean the same thing in both when you delete
      # and does a failed tokenizer actually do anything

for index, dataset_tensors in enumerate(comments_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    comments_tensors_data[index].append(text)

#for index, dataset_tensors in enumerate(posts_datasets_to_use):
#  for index1, text in enumerate(dataset_tensors):
#    encoded_input = tokenizer(text, return_tensors="pt", padding=True)
#    tensors_data[index].append(encoded_input)

In [20]:
print(post_tensors_data[0][0])

<bos> ###You may already know this, but college students with .edu email accounts are eligible for many online perks. I found this list from an AskReddit post a year back and found it surprisingly useful, although a bit outdated. I updated it and narrowed it down to the most worthwhile offers, but you can see the all the responses here: [**AskReddit**]([url link]  **[Amazon Prime]([url link] *6 month free trial, 50% off after trial* * Free unlimited photo storage * FREE Two-Day shipping with Amazon Prime * Watch unlimited movies and TV shows with Prime Instant Video * Shop Amazon Lightning Deals and MyHabit.com * A Prime membership includes over 500,000 free ebooks **[Autodesk]([url link] *Free 3D Design, Engineering  Entertainment Software* * AutoCAD * Maya (3D animation software) * 3ds Max * Revit * Fusion 360 Ultimate * Inventor Professional **[Microsoft Dreamspark]([url link] *Free Microsoft software* * Visual Studio Community 2013 (make applications for PC, Android, or iPhone) * W

In [21]:

GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO = "448 Group: Jedi Order/models/Iowa"
PATH_TO_PUT = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(os.listdir(PATH_TO_PUT))

448 Group: Jedi Order/models/Iowa
['train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'checkpoints-Iowa']


In [22]:
train_post_path = os.path.join(PATH_TO_PUT, "train_post_set.pickle")
val_post_path = os.path.join(PATH_TO_PUT, "val_post_set.pickle")
test_post_path = os.path.join(PATH_TO_PUT, "test_post_set.pickle")
train_com_path = os.path.join(PATH_TO_PUT, "train_com_set.pickle")
val_com_path = os.path.join(PATH_TO_PUT, "val_com_set.pickle")
test_com_path = os.path.join(PATH_TO_PUT, "test_com_set.pickle")

pickle.dump(posts_datasets_to_use[0], open(train_post_path, "wb"))
pickle.dump(posts_datasets_to_use[1], open(val_post_path, "wb"))
pickle.dump(posts_datasets_to_use[2], open(test_post_path, "wb"))
pickle.dump(comments_datasets_to_use[0], open(train_com_path, "wb"))
pickle.dump(comments_datasets_to_use[1], open(val_com_path, "wb"))
pickle.dump(comments_datasets_to_use[2], open(test_com_path, "wb"))

In [23]:
posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
posts_datasets_to_use[1] = pickle.load(open(val_post_path, "rb"))
posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
comments_datasets_to_use[1] = pickle.load(open(val_com_path, "rb"))
comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [24]:
import tensorflow
### dataset.py ###
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, dataset_type: str = 'Train'):
    super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.posts)

  def get_labels(self) -> pd.Series:
    return self.comments
  
  def __getitem__(self, idx):
    post_encoding = self.posts[idx]
    comment_encoding = self.comments[idx]
    if self.dataset_type == 'Train':
      token = self.tokenizer.encode(post_encoding + 'TL;DR' + comment_encoding, return_tensors="pt", 
                                    padding="max_length", truncation=True)
    return token

In [25]:
print(post_tensors_data[0][0])
print(comments_tensors_data[0][0])

<bos> ###You may already know this, but college students with .edu email accounts are eligible for many online perks. I found this list from an AskReddit post a year back and found it surprisingly useful, although a bit outdated. I updated it and narrowed it down to the most worthwhile offers, but you can see the all the responses here: [**AskReddit**]([url link]  **[Amazon Prime]([url link] *6 month free trial, 50% off after trial* * Free unlimited photo storage * FREE Two-Day shipping with Amazon Prime * Watch unlimited movies and TV shows with Prime Instant Video * Shop Amazon Lightning Deals and MyHabit.com * A Prime membership includes over 500,000 free ebooks **[Autodesk]([url link] *Free 3D Design, Engineering  Entertainment Software* * AutoCAD * Maya (3D animation software) * 3ds Max * Revit * Fusion 360 Ultimate * Inventor Professional **[Microsoft Dreamspark]([url link] *Free Microsoft software* * Visual Studio Community 2013 (make applications for PC, Android, or iPhone) * W

In [26]:
#print(len(tensors_data[index][0]["input_ids"][0]))
#print(len(tensors_data[index][0]["attention_mask"][0]))
#print(encoded_input["input_ids"].shape[1])
#print(encoded_input["attention_mask"].shape[1])

#train_to = zip(post_tensors_data[0], comments_tensors_data[0])
train_data = RedditDataset(post_tensors_data[0], comments_tensors_data[0])
#val_to = zip(post_tensors_data[1], comments_tensors_data[1])
val_data = RedditDataset(post_tensors_data[1], comments_tensors_data[1])
#test_to = zip(post_tensors_data[2], comments_tensors_data[2])
test_data = RedditDataset(post_tensors_data[2], comments_tensors_data[2])
#print(len(comments_tensors_data[0]))
#print(len(posts_datasets_to_use[0]))
#print(len(comments_datasets_to_use[0]))

In [27]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
#from transformers import TextDataset
#print(len(train_dataset_to_use))
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Iowa/checkpoints-Iowa")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    # do_train=True,
    num_train_epochs=1, # change this back to 2 
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy = "steps", #comment this
    eval_steps = 200,
    save_steps= 800,
    warmup_steps = 200,
    load_best_model_at_end = True, #comment this
    prediction_loss_only = True,
    )
# try and fix the above params as well
# convert dataset_to_use from a dictionary to a dataset type so that we can use it in the trainer
train_dataset_to_use_converted = torch.utils.data.DataLoader(train_dataset_to_use)
#print(type(train_dataset_to_use_converted))
test_dataset_to_use_converted = torch.utils.data.DataLoader(test_dataset_to_use)

#early_stop = EarlyStoppingCallback(5, 1.0) #comment this

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_data,
    eval_dataset=test_data
)

# depends on whether we want to use Trainer or not. I think using Trainer would be better if we get it to work,
# but I'm not sure how feasible it is. Additionally manual is kinda what we did in class
# ask about batching our data, and ask about num_labels. Batching our data in that dataloader part. like how to iterate over batches of data.
# idk if need to batch

In [28]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
200,No log,3.825095
400,No log,3.497840


TrainOutput(global_step=477, training_loss=11.049341997510481, metrics={'train_runtime': 198.2564, 'train_samples_per_second': 2.406, 'train_steps_per_second': 2.406, 'total_flos': 249272598528000.0, 'train_loss': 11.049341997510481, 'epoch': 1.0})

In [29]:
trainer.save_model()

VVVVVVVVrun if needed

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


^^^^^^run if needed 

In [30]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
post_in_one = "I saw a squirrel today. I feel blessed. So fluffy. I saw it eat cute nuts."
post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_with_bos = "<bos>" + post_in + " TL;DR"
encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
    )
generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))
# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [31]:
def make_sent(input):
    output = ""
    end_things = ".!?"
    last_end = 0
    i = 0
    while i < len(input):
        if input[i].isascii() or "'":
            if input[i] in end_things:
                output = output + input[i] + "\n"
                i = i + 1
                last_end = i
                if i + 1 < len(input):
                    if input[i] == " ":
                        i = i + 1
                        if i < len(input):
                            while input[i] == " ":
                                i = i + 1
                                if i >= len(input):
                                    break
                            output = output + " "  
            else:
                if i + 1 == len(input):
                    if input[i] == ",":
                        output = output + "."
                    else:
                        output = output + input[i]
                        output = output + "."
                else:
                    output = output + input[i]
                i = i + 1
    return output

In [32]:
#https://python.plainenglish.io/use-python-to-split-sentences-into-separate-lines-17a577f9ddb3
print("Post: ")

post_out_now = make_sent(post_in)
print(post_out_now)

i = 1
for x in generated_sequences:
    out = x.split("TL;DR")
    comment = out[1]
    print("Comment {}: ".format(i))
    comment_out_now = make_sent(comment)
    print(comment_out_now)
    i = i+1
    print('\n')

Post: 
I want to go to hotel management school for rural bed and breakfasts.
 What experiences or classes should I seek out?

Comment 1: 
 You don't need to have any experience from a hotel.
 There are some great perks at the hotel and if you get into a band then there is another place to choose from.
 You should definitely bring your own band if you don't have the money, which sounds like a better deal than just having the company come for you in and out.
 For example, I'm.


Comment 2: 
 If you're not on campus, check off the website and see if there's a way to find a private school for you.
 I want to go to a private school to find my first year.
 I don't want to take the two year class so I'll just be looking at private schools.



Comment 3: 
 School is what I go to school for in the morning.
 I was in the morning and the rest of the classes were in the afternoon.





Validation

In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=7f0efce1aa3e4f9eec01c463e3f4eded48662b4508a73e0e449218f040b9029b
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [33]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  post_with_bos = "<bos>" + post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(comments_datasets_to_use[1])
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(comments_datasets_to_use[1]):
  val_dataset[cnt]['reference_summary'] = item

for cnt, post in enumerate(post_tensors_data[1]):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: Could someone please give me a description of the different dorms at Iowa? I'm going there next year, and I have no idea what the dorms are like. It would be much appreciated [:    Yes, I am coming to Iowa and they are quite well structured. There are a lot of very good things about this university but they have a really good chemistry section and it's a really good atmosphere at all times.
Reference COMMENTS:<bos> Hopefully you've already seen this: [url link] East side (main campus side): Burge has recently been expanded and so there are some brand new suites (shared bathroom between two two-person rooms) that are very nice. The east-side cafeteria is also there, which is convenient so that you don't have to go outside. There is also a C-store (like a convenience store that you can use hawk dollars at). The floors with suites are coed, otherwise it is separated by floor. The doors here stay unlocked longer hours than other dorms because of the dining hall.  Daum i

In [ ]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.21349895263685129, 'precision': 0.20297952387719986, 'fmeasure': 0.18679155466932557}
Overall ROUGE-2 scores: {'recall': 0.027422020991515524, 'precision': 0.023600854845333735, 'fmeasure': 0.02180601138293059}
Overall ROUGE-L scores: {'recall': 0.18797228885841283, 'precision': 0.17905493642283263, 'fmeasure': 0.1646261765607726}


Word2Vec Validation

In [42]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

def generate_comments(post):
  # print(post)
  post_with_bos = "<bos>" + post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=10
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

dflen = len(comments_datasets_to_use[1])
df = [[] for _ in range(dflen)]

for cnt, item in enumerate(comments_datasets_to_use[1]):
  df[cnt].append(item)

for cnt, post in enumerate(post_tensors_data[1]):
  # print(post)
  generated_sequences = generate_comments(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    df[cnt].append(sent)
    # print(sent)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your in

In [43]:
# tokenize and pad every document to make them of the same size

for documents in df:
  documents_df=pd.DataFrame(documents,columns=['documents'])

  # removing special characters and stop words from the text
  stop_words_l=stopwords.words('english')
  documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

  tfidfvectoriser=TfidfVectorizer()
  tfidfvectoriser.fit(documents_df.documents_cleaned)
  tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

  pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T).toarray()
  pairwise_differences=euclidean_distances(tfidf_vectors)

  def most_similar(doc_id,similarity_matrix,matrix):
      print (f'Original Comment: {documents_df.iloc[doc_id]["documents"]}')
      print ('\n')
      print ('Similar Comments:')
      if matrix=='Cosine Similarity':
          similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
      elif matrix=='Euclidean Distance':
          similar_ix=np.argsort(similarity_matrix[doc_id])
      for ix in similar_ix:
          if ix==doc_id:
              continue
          print('\n')
          print (f'Generated Comment: {documents_df.iloc[ix]["documents"]}')
          print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

  most_similar(0,pairwise_similarities,'Cosine Similarity')
  # most_similar(0,pairwise_differences,'Euclidean Distance')  
  break

Document: <bos> I lived in Hillcrest for two years 2000-2002. I would suggest leaving your door open and be friendly to those in your hall. When I lived there a lot of students were busy in sports. But once I had Mario tennis/kart playing I made friends in the hall playing together. <eos> 


Similar Comments:


Actual Comment: I'm a transfer student starting my first year and I was places on the second floor of Hillcrest. Anyone else living in that dorm? So far all of my friends are living in Burge.    I lived there and I could transfer to the Iowa Hawkeye program. I was so excited to find out about this class I was hoping it would be a lot more fun. If this class can have a lot of students I would be interested in the class. Anyone interested? Please let me know if they could have more fun or not so much better than you do.
Cosine Similarity : 0.10664013924556973


Actual Comment: I'm a transfer student starting my first year and I was places on the second floor of Hillcrest. Anyone e

In [1]:
# tokenize and pad every document to make them of the same size
!pip install Keras-Preprocessing
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

for documents in df:
  documents_df=pd.DataFrame(documents,columns=['documents'])

  # removing special characters and stop words from the text
  stop_words_l=stopwords.words('english')
  documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(documents_df.documents_cleaned)
  tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
  tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
  vocab_size=len(tokenizer.word_index)+1
  print (tokenized_paded_documents[0])

  # loading pre-trained embeddings, each word is represented as a 300 dimensional vector
  import gensim
  W2V_PATH="GoogleNews-vectors-negative300.bin.gz"
  model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

  # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
  embedding_matrix=np.zeros((vocab_size,300))
  for word,i in tokenizer.word_index.items():
      if word in model_w2v:
          embedding_matrix[i]=model_w2v[word]
  # creating document-word embeddings
  document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))
  for i in range(len(tokenized_paded_documents)):
      for j in range(len(tokenized_paded_documents[0])):
          document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
  document_word_embeddings.shape

  # calculating average of word vectors of a document weighted by tf-idf
  document_embeddings=np.zeros((len(tokenized_paded_documents),300))
  words=tfidfvectoriser.get_feature_names()
  for i in range(len(document_word_embeddings)):
      for j in range(len(words)):
          document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
  print (document_embeddings.shape)
  pairwise_similarities=cosine_similarity(document_embeddings)
  pairwise_differences=euclidean_distances(document_embeddings)
  most_similar(0,pairwise_similarities,'Cosine Similarity')
  break


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 891.1 kB/s eta 0:00:00


NameError: ignored

Sentiment

In [ ]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.1 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [ ]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  print(sent_gt)
  print(gt_label)
  print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  print(sent_generated_comment)
  print(generated_comment_label)
  if idx == 15:
    break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

Reference comments: <bos> I lived in Hillcrest for two years 2000-2002. I would suggest leaving your door open and be friendly to those in your hall. When I lived there a lot of students were busy in sports. But once I had Mario tennis/kart playing I made friends in the hall playing together. <eos> 
{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'compound': 0.8641}
neutral
Generated comments: I'm a transfer student starting my first year and I was places on the second floor of Hillcrest. Anyone else living in that dorm? So far all of my friends are living in Burge.    "That dorm is so far from the Burge dorm but I'll be staying in those places as long as I can. You do that with a college dorm, but I'm living with a friend for my fifth year in that dorm. I'm going to bring my friend (which was the first floor dorm I had the choice to choose) to the dorm.
{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.8922}
neutral
Reference comments: <bos> thank you for this. Does it accept all file t

In [ ]:
for item in comments_datasets_to_use[1]:
  print(item)

<bos> I lived in Hillcrest for two years 2000-2002. I would suggest leaving your door open and be friendly to those in your hall. When I lived there a lot of students were busy in sports. But once I had Mario tennis/kart playing I made friends in the hall playing together. <eos> 
<bos> thank you for this. Does it accept all file types (like pdfs) or only word documents? pdf/doc/docx :) <eos> 
<bos> I graduated from U of I in 2005 with a BS in CS. My information might be a bit dated. We concentrated heavily on Java for any actual implementation of theory. We spent time on algorithms, OS theory, programming language theory, and low-level (machine) theory. Fairly well balanced for a good grounding. I highly recommend that you get involved in programming on your own time. Find a project that interests you and send PRs to the project on GitHub.  Don't be afraid to use a language that interests you. I use Perl professionally, but it wasn't really ever even mentioned in lectures. <eos> 
<bos>